In [1]:
#!!! Fix data structure for features_lst -> interpreted as string 
# should be list of str elems

In [2]:
# for colab compatibility
!pip install shap
!pip install ppscore
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
%run data_load_wrapper.ipynb

In [4]:
# rip - bug1: feat_ind refers to local instead of global index ordering in function model_fit
# rip - bug2: stats dataframe is off by one for mse_test and train - now we have a shifted column
# 0th iteration replaced with full_set
#
# refactor1: get_metric
# refactor2: into class MetaReducer(dataset, imp_types, models) 
# refactor3: create a "smart" calculation execution for feature reduction
# we can pool the "votes" for feature reduction and only retrain the model once per vote class
# (feature to reduce), might save decent amount of time for large datasets 
# (cases where model training takes significantly longer than importance calculation)
# current: retrain the model every time for each {imp_type} - not as important right now, since
# model training in the worst case takes 3s, feature importance calculation is the bottleneck
""" 
dataset - sklearn_dataset object or tuple/list of tuples of (x, y)
imp_types - list of importance types
supported types: ['gini', 'pimp', 'shap', 'pps', 'pearson', 'spearman', 'kendall']
models - list of machine learning models from sklearn lib
[which ones? tbd]
debug = {False, True} display debug_info into the dataframe
return -> dataframe of reduction stats
""";

In [2]:
from tqdm.notebook import trange
import sklearn.datasets as data
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn import *
from sklearn.feature_selection import *
from sklearn.datasets import make_regression
from sklearn.utils import Bunch
import matplotlib as plt
import seaborn as sns
import shap
import numpy as np
import ppscore as pps
from datetime import datetime
import xgboost

In [17]:
df.head(11)

,iteration,dropped_feature,feat_ind,feat_name,mse_train,mse_test,model,imp_type,dataset,features_lst,random_state,data_split
0,0,full_set,5,x_5,0.0,3664.259545,DecisionTreeRegressor,gini,regr_features_10_samples_100_seed_0,"Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5...",0,0
1,1,x_5,8,x_8,0.0,3286.260852,DecisionTreeRegressor,gini,regr_features_10_samples_100_seed_0,"Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_6...",0,0
2,2,x_8,1,x_1,0.0,2988.037484,DecisionTreeRegressor,gini,regr_features_10_samples_100_seed_0,"Index(['x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_6...",0,0
3,3,x_1,4,x_4,0.0,3367.877516,DecisionTreeRegressor,gini,regr_features_10_samples_100_seed_0,"Index(['x_0', 'x_2', 'x_3', 'x_4', 'x_6', 'x_7...",0,0
4,4,x_4,0,x_0,0.0,3247.106739,DecisionTreeRegressor,gini,regr_features_10_samples_100_seed_0,"Index(['x_0', 'x_2', 'x_3', 'x_6', 'x_7', 'x_9...",0,0
5,5,x_0,7,x_7,0.0,3476.040319,DecisionTreeRegressor,gini,regr_features_10_samples_100_seed_0,"Index(['x_2', 'x_3', 'x_6', 'x_7', 'x_9'], dty...",0,0
6,6,x_7,2,x_2,0.0,3246.404391,DecisionTreeRegressor,gini,regr_features_10_samples_100_seed_0,"Index(['x_2', 'x_3', 'x_6', 'x_9'], dtype='obj...",0,0
7,7,x_2,9,x_9,0.0,1647.305356,DecisionTreeRegressor,gini,regr_features_10_samples_100_seed_0,"Index(['x_3', 'x_6', 'x_9'], dtype='object')",0,0
8,8,x_9,6,x_6,0.0,1093.167821,DecisionTreeRegressor,gini,regr_features_10_samples_100_seed_0,"Index(['x_3', 'x_6'], dtype='object')",0,0
9,9,x_6,3,x_3,0.0,8390.729297,DecisionTreeRegressor,gini,regr_features_10_samples_100_seed_0,"Index(['x_3'], dtype='object')",0,0


In [4]:
df = pd.read_csv('regression_synthethic_2021-Jul-26-17:28:33.csv')

In [6]:
def sklearn_to_df(dataset):
    df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
    y = dataset.target
    return df, y

def get_feat_imp(imp_type, m, x_train, y_train, random_state):
    imp_type_dict = {
        'mutual_info' : 'mutual_info_regression(x_train, y_train)',
        'f_test' : 'f_regression(x_train, y_train)[0]',       
        'gini' : "m.feature_importances_",
        'pimp' : f"""permutation_importance(m, x_train, y_train, n_repeats=5, 
        random_state={random_state}).importances_mean""",
        'shap' : "abs(shap.TreeExplainer(m).shap_values(x_train).mean(axis=0))",
        
        'pps' : """(pps.predictors(x_train.join(y_train), 
                                'target', sorted=False, output='df').model_score
                                )""",
        
        'pearson' : "abs(x_train.join(y_train).corr(method='pearson').target[:-1])",
        'spearman': "abs(x_train.join(y_train).corr(method='spearman').target[:-1])",
        'kendall' : "abs(x_train.join(y_train).corr(method='kendall').target[:-1])"
    }
#     print(eval(imp_type_dict.get(imp_type)))
    return eval(imp_type_dict.get(imp_type))

def reduce_dataset(x_train, x_test, drop_index):
    x_train = x_train.drop(x_train.columns[drop_index], axis=1)
    x_test = x_test.drop(x_test.columns[drop_index], axis=1)
    return x_train, x_test

def eval_reduced(x_train, y_train, x_test, y_test, m, imp_type, 
                 dataset, dataset_str, random_state, dataset_state):
    reduce_stats = []
    columns_init = np.array(dataset.feature_names)
#     dataset_name = dataset.filename.split('/')[-1]
    for i in range(len(x_train.columns)):
        mse_train, mse_test, drop_index, feat = model_fit(x_train, y_train, 
                                                     x_test, y_test, m, imp_type, random_state)
        global_index = np.where(columns_init==feat);
        global_index = global_index[0][0]
        reduce_stats.append((i, global_index, feat, 
                             mse_train, mse_test, 
                             # get class name, remove trash
                             str(m.__class__).split('.')[-1].replace("\'", '').strip('>'), 
                             imp_type, dataset_str, x_train.columns, random_state, dataset_state))
        x_train, x_test = reduce_dataset(x_train, x_test, drop_index)
    return reduce_stats

def model_fit(x_train, y_train, x_test, y_test, m, imp_type, random_state):
#     print(x_train.shape, y_train.shape)
#     refactor1 into function get_metric(type={train, test})
    m.fit(x_train, y_train)
    res_train = m.predict(x_train)
    res_test = m.predict(x_test)
    mse_test = metrics.mean_squared_error(y_test, res_test)
    mse_train = metrics.mean_squared_error(y_train, res_train)
    # get the index for feature with lowest global feature importance
    imp = get_feat_imp(imp_type, m, x_train, y_train, random_state)
    argmin_fi_index = imp.argmin()
    feat = x_train.columns[argmin_fi_index]
    return mse_train, mse_test, argmin_fi_index, feat

def eval_reduced_one_shot(x_train, y_train, x_test, y_test, m, imp_type, 
                 dataset, dataset_str, random_state, dataset_state):
    
    reduce_stats = []
    columns_init = np.array(x_train.columns)
    m.set_params(random_state = random_state)
    m.fit(x_train, y_train)
    fi_scores = get_feat_imp(imp_type, m, x_train, y_train, random_state)
#     print(f'imp: {imp_type} \n model: {m} \n x_shape: {x_train.shape} \n y_shape: {y_train.shape}')
#     print(f'arr_scores: {fi_scores} \n shape: {fi_scores.shape}')
    
    for i in range(len(x_train.columns)):
        mse_train, mse_test, drop_index, feat = model_fit_one_shot(x_train, y_train, 
                                                     x_test, y_test, m, fi_scores, random_state)
        global_index = np.where(columns_init==feat);
        global_index = global_index[0][0]
#         print(global_index)
#         print(fi_scores)
        reduce_stats.append((i, global_index, feat, 
                             mse_train, mse_test, 
                             # get class name, remove trash
                             str(m.__class__).split('.')[-1].replace("\'", '').strip('>'), 
                             imp_type, dataset_str, x_train.columns, random_state, dataset_state))
        x_train, x_test = reduce_dataset(x_train, x_test, drop_index)
#         print(f'iter {i}')
#         print(f'local_index: {drop_index}')
#         print(f'global_index: {global_index}, \n fi before: \n {fi_scores}')
#         print(fi_scores.shape)
        fi_scores = np.array(fi_scores)
        fi_scores = np.delete(fi_scores, drop_index)
#         print(f'global_index: {global_index}, \n fi after: \n {fi_scores}')              
    return reduce_stats

def model_fit_one_shot(x_train, y_train, x_test, y_test, m, fi_scores, 
                       random_state):
#     print(x_train.shape, y_train.shape)
#     refactor1 into function get_metric(type={train, test})
    m.set_params(random_state=random_state)
    m.fit(x_train, y_train)
    res_train = m.predict(x_train)
    res_test = m.predict(x_test)
    mse_test = metrics.mean_squared_error(y_test, res_test)
    mse_train = metrics.mean_squared_error(y_train, res_train)
    # get the index for feature with lowest global feature importance
    imp = fi_scores
    argmin_fi_index = imp.argmin()
    
    feat = x_train.columns[argmin_fi_index]
    return mse_train, mse_test, argmin_fi_index, feat

In [7]:
def get_stats(imp_types, models, datasets, debug=False, random_states=[0], dataset_states=[0]):
    results = []
    datasets_str = list(datasets.keys())
    datasets_data = datasets.items()
    random_state = 0
    for ds in trange(len(dataset_states), desc='Data state progress'):
        dataset_state = dataset_states[ds]
        for r in trange(len(random_states), desc='Model random state progress'):
            random_state = random_states[r]

            for j in trange(len(datasets_str), desc=f'Dataset progress'):
                dataset_str = datasets_str[j]
                dataset = datasets.get(datasets_str[j])
                df, y = sklearn_to_df(dataset)
                x_train, x_test = model_selection.train_test_split(df, test_size=0.3, 
                                                                   random_state=dataset_state)
                columns_init = x_train.columns
                y_train = pd.DataFrame(y[x_train.index], index=x_train.index, columns=['target'])
                y_test = pd.DataFrame(y[x_test.index], index=x_test.index, columns=['target'])

                for k in trange(len(models), desc=f'Model progress for dataset {j}'):
                    m = models[k]
                    m.set_params(random_state=random_state)
                    m.fit(x_train, y_train.target)
                    res_test = m.predict(x_test)
                    res_train = m.predict(x_train)
                    mse_test = metrics.mean_squared_error(y_test, res_test)
                    mse_train = metrics.mean_squared_error(y_train, res_train)

                    for i in trange(len(imp_types), desc=f'Importance type progress for model {k}'):
                        if imp_types[i] in ['shap', 'pimp', 'pearson', 'spearman', 'kendall',
                                            'pps', 'mutual_info', 'f_test']:
                            result = eval_reduced_one_shot(x_train, y_train, x_test, y_test, 
                                                           m, imp_types[i], dataset, dataset_str, 
                                                           random_state, dataset_state)
                        else:
                            result = eval_reduced(x_train, y_train.target, x_test, y_test, 
                                                  m, imp_types[i], dataset, dataset_str, 
                                                  random_state, dataset_state)
                        results.extend(result)
                df_res = pd.DataFrame(results, columns=['iteration', 'feat_ind', 'feat_name',
                                                    'mse_train', 'mse_test','model', 'imp_type', 
                                                    'dataset', 'features_lst', 
                                                    'random_state', 'data_split'])
        # shift features by 1 row, replace 0th iteration with 'full_set'
        df_res['dropped_feature'] = df_res['feat_name'].shift(1)
        df_res['dropped_feature'] = df_res['dropped_feature'].where(df_res['iteration'] != 0, 'full_set')
        if debug==True:
            repr_lst = ['iteration', 'dropped_feature', 'feat_ind', 'feat_name',
                        'mse_train', 'mse_test','model', 'imp_type', 
                        'dataset', 'features_lst','random_state', 'data_split']
        else:
            repr_lst = ['iteration','dropped_feature','mse_train',
         'mse_test','model','imp_type', 'dataset', 'random_state', 'data_split']
        df_res = df_res[repr_lst]
    return df_res

In [8]:
def coerce_synthethic_datasets(samples, features, percent_informative, random_state):
    data, target = make_regression(n_samples=samples, n_features=features, 
                                       n_informative=int(percent_informative*features),
                                       random_state=random_state
                                      )
    feature_names = [f'x_{i}' for i in range(features)]
    # data, target, feature_names
    data_bunch = Bunch(
        data=data,
        target=target,
        feature_names=feature_names,
        informative_features=feature_names[:int(percent_informative*features)]
                        )
    return data_bunch

In [9]:
datasets = {}
imp_types = [
             'gini',
             'mutual_info',
             'f_test',
             'pimp', 
             'shap', 
             'pearson', 
             'spearman', 
             'kendall', 
             'pps'
            ]
models = [
          tree.DecisionTreeRegressor(random_state=0), 
          ensemble.RandomForestRegressor(random_state=0, n_jobs=3, verbose=0), 
          xgboost.XGBRegressor(random_state=0, n_jobs=3, verbosity=0)
         ]

random_states = [i for i in range(1)]
dataset_states = [i for i in range(1)]

features = [10, 100, 100]
samples = [100, 1000]
synthethic_states = [i for i in range(10)]
percent_informative = 0.2

tuples = []
for i in features:
    for j in samples:
        if features != samples:
            for k in synthethic_states:            
                tuples.append((i,j,k))
datasets = {}

In [10]:
for feat, samples, random_seed in tuples:
    datasets[f'regr_features_{feat}_samples_{samples}_seed_{random_seed}'] = coerce_synthethic_datasets(samples, feat, 0.2, random_seed)

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
reg_data = get_stats(imp_types, models, datasets, debug=True, 
                   random_states=random_states, dataset_states=dataset_states)

Data state progress:   0%|          | 0/1 [00:00<?, ?it/s]

Model random state progress:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset progress:   0%|          | 0/40 [00:00<?, ?it/s]

Model progress for dataset 0:   0%|          | 0/3 [00:00<?, ?it/s]

Importance type progress for model 0:   0%|          | 0/9 [00:00<?, ?it/s]

...

Importance type progress for model 2:   0%|          | 0/9 [00:00<?, ?it/s]

Model progress for dataset 39:   0%|          | 0/3 [00:00<?, ?it/s]

Importance type progress for model 0:   0%|          | 0/9 [00:00<?, ?it/s]

Importance type progress for model 1:   0%|          | 0/9 [00:00<?, ?it/s]

Importance type progress for model 2:   0%|          | 0/9 [00:00<?, ?it/s]

In [13]:
# export results to local dir as csv file
df_res.to_csv(f'regression_synthethic_{datetime.today().strftime("%Y-%b-%d-%H:%M:%S")}.csv', index=False)

In [14]:
# import seaborn as sns

# plot = sns.lineplot(x='iteration', y='mse_test', hue='imp_type',
#             data=df_res.query('model == "XGBRegressor" and dataset == "diabetes"')
#             )